In [10]:
from __future__ import print_function, division

import torch
from sklearn import metrics
from torch import nn, Tensor
from torchvision import transforms, models
import matplotlib.pyplot as plt
from PIL import Image

from DataReaders.ChenAudiosetDataset import ChenAudiosetDataset
from DataReaders.Test_fgo import Test_fgo
from MultiTask.HardSharing import HardSharing
from MultiTask.pytorch_multitask_example.multi_output_model import multi_output_model
from Tasks.ConcatTaskDataset import ConcatTaskDataset
from Tasks.Task import Task
from Tasks.TaskDataset import TaskDataset
import os

print(os.getcwd())
plt.ion()  # interactive mode
import pandas as pd
from sklearn.model_selection import train_test_split

C:\Users\mrKC1\PycharmProjects\Thesis\Tests


In [11]:
# own model
# Important: the tasks in task_list has to be the same as the order in concatTaskDataset
# Required:
# ConcatTaskDataset
# ->    list of TaskDataset
# ->    inputs: list of size nr_instances, flatened tensors
# ->    targets: tensors of right outputs
# ->    name: name as task, consistent with task_list
# X_input_size: input nodes amount
# task_list: list of object Task:
# ->    name: name as task, consistent with concattaskdataset
# ->    output_labels: list of string with output labels

# test_fgo = Test_fgo()
# training_dataset = test_fgo.toTaskDataset()
#
# model = HardSharing(test_fgo.get_input_nodes(), 128, 3, training_dataset.get_task_list())
# model = model.cuda()

In [12]:

# multitask_example model
# X_loaded = [Image.open('Datasets/images/' + x) for x in X_train]
# X_loaded = [x.convert('RGB') for x in X_loaded]
# X_loaded = [data_transforms['train'](x) for x in X_loaded]
# X_input_size = X_loaded[0].shape[0]
#
# training_dataset = ConcatTaskDataset(
#     [TaskDataset(inputs=X_loaded, targets=gender_train, name='gender'),
#      TaskDataset(inputs=X_loaded, targets=region_train, name='region'),
#      TaskDataset(inputs=X_loaded, targets=fighting_style_train, name='fighting'),
#      TaskDataset(inputs=X_loaded, targets=alignment_train, name='alignment'),
#      TaskDataset(inputs=X_loaded, targets=color_train, name='colors')]
# )
#
# dd = .1
# model_ft = models.resnet50(pretrained=True)
# print(model_ft)
# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 512)
# model = multi_output_model(model_ft, dd, [gender_nodes, region_nodes, fighting_nodes, alignment_nodes, color_nodes])
# model = model.cuda()

In [13]:

chendata = ChenAudiosetDataset()

done


In [14]:
chenTaskDataset = chendata.toTaskDataset()
training_dataset = ConcatTaskDataset([chenTaskDataset])
task_list = training_dataset.get_task_list()

model = HardSharing(len(chenTaskDataset.inputs[0]), 128, 3, task_list)
model = model.cuda()

In [15]:
from Training.Training import Training
print('started')
Training.run_gradient_descent(
    model=model,
    concat_dataset=training_dataset,
    batch_size=16,
    num_epochs=50
    # learning_rate=,
    # weight_decay=,
)

started
Epoch 0
Not updated parameters?
False
Epoch 1
Not updated parameters?
False
Epoch 2
Not updated parameters?
False
Epoch 3
Not updated parameters?
False
Epoch 4
Not updated parameters?
False
Epoch 5
Not updated parameters?
False
Epoch 6
Not updated parameters?
False
Epoch 7
Not updated parameters?
False
Epoch 8
Not updated parameters?
False
Epoch 9
Not updated parameters?
False
Epoch 10
Not updated parameters?
False
Epoch 11
Not updated parameters?
False
Epoch 12
Not updated parameters?
False
Epoch 13
Not updated parameters?
False
Epoch 14
Not updated parameters?
False
Epoch 15
Not updated parameters?
False
Epoch 16
Not updated parameters?
False
Epoch 17
Not updated parameters?
False
Epoch 18
Not updated parameters?
False
Epoch 19
Not updated parameters?
False
Epoch 20
Not updated parameters?
False
Epoch 21
Not updated parameters?
False
Epoch 22
Not updated parameters?
False
Epoch 23
Not updated parameters?
False
Epoch 24
Not updated parameters?
False
Epoch 25
Not updated parame

HardSharing(
  (model): Sequential(
    (hard_sharing): FFNN(
      (hidden): ModuleList(
        (0): Linear(in_features=182, out_features=128, bias=True)
        (1): Linear(in_features=128, out_features=128, bias=True)
        (2): Linear(in_features=128, out_features=128, bias=True)
        (3): Linear(in_features=128, out_features=128, bias=True)
      )
      (relu): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (task_specific): TaskDependentLayers(
      (task_nets): ModuleList(
        (0): Linear(in_features=128, out_features=25, bias=True)
      )
    )
  )
)

In [16]:
train_loader = torch.utils.data.DataLoader(
            training_dataset,
            # sampler=BalancedBatchSchedulerSampler(dataset=concat_dataset,
            #                                       batch_size=batch_size),
            batch_size=1,
            shuffle=True,
            num_workers=0)

model.eval()
outputs = []
trues = []
for inputs, targets, names in train_loader:
    out = model(inputs.cuda())
    outputs.append(out)
    true = targets
    trues.append(true)
outputs = torch.cat(outputs, dim=0).detach().cpu()
trues = torch.cat(trues, dim=0).detach().cpu()

In [17]:
fntp = trues.sum()
tpfp = (outputs >= 0.5).sum()
tp = ((outputs[trues > 0.5] >= 0.5) == trues[trues > 0.5]).sum()
precision = tp/tpfp
recall = tp/fntp
print('True positives: {}'.format(tp))
print('Total classified as positive: {}'.format(tpfp))
print('Total positive cases: {}'.format(fntp))
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F1 Score: {}'.format(2*(precision*recall)/(precision+recall)))
print()

for i in range(trues.shape[1]):
    trues_y = trues[:, i]
    outputs_y = outputs[:, i]
    fntp = trues_y.sum()
    tpfp = (outputs_y >= 0.5).sum()
    tp = ((outputs_y[trues_y > 0.5] >= 0.5) == trues_y[trues_y > 0.5]).sum()
    print('Label: '+ chenTaskDataset.task.output_labels[i])
    print('True positives for {}: {}'.format(i, tp))
    print('Total classified as positive for {}: {}'.format(i, tpfp))
    print('Total positive cases for {}: {}'.format(i, fntp))
    print('Precision for {}: {}'.format(i, tp/tpfp))
    print('Recall for {}: {}'.format(i, tp/fntp))
    print()

True positives: 1967
Total classified as positive: 2695
Total positive cases: 9356.0
Precision: 0.7298701405525208
Recall: 0.210239418554938
F1 Score: 0.32644593916491194

Label: Typing
True positives for 0: 2
Total classified as positive for 0: 2
Total positive cases for 0: 308.0
Precision for 0: 1.0
Recall for 0: 0.006493506493506494

Label: Pink noise
True positives for 1: 101
Total classified as positive for 1: 167
Total positive cases for 1: 315.0
Precision for 1: 0.6047903895378113
Recall for 1: 0.32063492063492066

Label: Throat clearing
True positives for 2: 0
Total classified as positive for 2: 0
Total positive cases for 2: 167.0
Precision for 2: nan
Recall for 2: 0.0

Label: Hubbub, speech noise, speech babble
True positives for 3: 0
Total classified as positive for 3: 0
Total positive cases for 3: 225.0
Precision for 3: nan
Recall for 3: 0.0

Label: Speech
True positives for 4: 0
Total classified as positive for 4: 0
Total positive cases for 4: 500.0
Precision for 4: nan
Rec

In [18]:
model.eval()
inp = torch.stack([chenTaskDataset.inputs[0], chenTaskDataset.inputs[1]]).cuda()
tgt = torch.stack([chenTaskDataset.__getitem__(0)[1], chenTaskDataset.__getitem__(1)[1]]).cuda()
out = model(inp)

print(out)
print((out >= 0.5) == tgt)


tensor([[5.0113e-03, 3.6083e-03, 4.1867e-02, 4.5969e-03, 9.9742e-02, 1.4168e-02,
         2.2154e-02, 1.6895e-01, 1.9300e-02, 9.4136e-03, 2.5174e-03, 2.7254e-01,
         9.0539e-02, 5.3343e-02, 8.4156e-03, 6.5776e-03, 7.2212e-05, 3.6127e-02,
         1.7849e-02, 8.7957e-05, 2.1319e-03, 5.0696e-03, 3.3034e-02, 1.0107e-02,
         7.3191e-03],
        [1.3281e-02, 5.3271e-02, 2.3084e-02, 3.9226e-02, 2.0886e-01, 1.7018e-01,
         1.0834e-01, 2.8347e-01, 6.7080e-02, 1.7305e-02, 1.8015e-02, 3.4521e-02,
         3.2050e-02, 5.1727e-02, 2.6902e-02, 2.9529e-02, 2.9227e-03, 8.6651e-02,
         3.0438e-02, 4.3660e-03, 6.8486e-02, 1.2346e-02, 1.7688e-02, 7.9590e-02,
         5.0720e-02]], device='cuda:0', grad_fn=<CatBackward>)
tensor([[ True,  True,  True,  True,  True,  True, False,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True, False,  True,